In [1]:
import keras
from keras.layers import Embedding, Input, Flatten, Dense, concatenate, merge, add
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import get_file
import numpy as np

Using TensorFlow backend.


In [2]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600901


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 86


using zero value in the dataset for padding

In [4]:
chars.insert(0, "\0")

In [5]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

Map from chars to indices and back again

In [6]:
char_indices = dict((c,i) for i, c in enumerate(chars))
indices_char = dict((i,c) for i, c in enumerate(chars))

idx will be the data we use for now own - it simply coverts all the characters to their index (based on the mapping above)

In [7]:
idx = [char_indices[c] for c in text]

In [8]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [9]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

## 3 Char model

#### Create inputs

create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [10]:
len(idx)

600901

In [11]:
cs = 3
c1_dat = [idx[i] for i in range(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-1-cs, cs)]


Our inputs

In [12]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

Our Output

In [13]:
y = np.stack(c4_dat[:-2])

In [14]:
test = [i for i in range(1,10)]

The first 4 inputs and outputs

In [15]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [16]:
y[:4]

array([30, 29,  1, 40])

In [17]:
x1.shape, y.shape

((200297,), (200297,))

The number of latent factors to create (i.e. the size of the embedding matrix)

In [18]:
n_fac = 42

Create inputs and embedding outputs for each of our 3 character inputs

In [19]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [20]:
c1_in, c1 = embedding_input("c1", vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Create and train model

Pick a size for our hidden state

In [21]:
n_hidden = 256

This is the 'green arrow' from our diagram - the layer operation from input to hidden.

In [22]:
dense_in = Dense(n_hidden, activation='relu')

our first hidden activation is simply this fucntion applied to the result of the embeeding of the frist character.

In [23]:
c1_hidden = dense_in(c1)

This is the 'orange arrow' from our diagram - the layer operation from hidden to hidden.

In [24]:
dense_hidden = Dense(n_hidden, activation='tanh')

Our second and third hidden activations sum up the previous hidden state (after applying dense_hidden) to the new input state.

In [25]:
c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = add([c2_dense, hidden_2])

In [26]:
c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = add([c3_dense, hidden_3])

This is the 'blue arrow' from our diagram - the layer operation from hidden to output

In [27]:
dense_out = Dense(vocab_size, activation='softmax')

The third hidden state is the input to our output layer.

In [28]:
c4_out = dense_out(c3_hidden)

In [29]:
model = Model([c1_in, c2_in, c3_in], c4_out)

In [30]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [62]:
model.optimizer.lr=0.0000001

In [79]:
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)

Epoch 1/28
200297/200297 [==============================] - 14s - loss: 3.0707    
Epoch 2/28
200297/200297 [==============================] - 13s - loss: 3.0700    
Epoch 3/28
200297/200297 [==============================] - 14s - loss: 3.0692    
Epoch 4/28
200297/200297 [==============================] - 13s - loss: 3.0685    
Epoch 5/28
200297/200297 [==============================] - 14s - loss: 3.0678    
Epoch 6/28
200297/200297 [==============================] - 13s - loss: 3.0670    
Epoch 7/28
200297/200297 [==============================] - 13s - loss: 3.0663    
Epoch 8/28
200297/200297 [==============================] - 13s - loss: 3.0655    
Epoch 9/28
200297/200297 [==============================] - 13s - loss: 3.0648    
Epoch 10/28
200297/200297 [==============================] - 13s - loss: 3.0641    
Epoch 11/28
200297/200297 [==============================] - 13s - loss: 3.0634    
Epoch 12/28
200297/200297 [==============================] - 13s - loss: 3.0626    
E

In [80]:
model.optimizer.lr=0.01

In [81]:
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)

Epoch 1/28
200297/200297 [==============================] - 14s - loss: 3.0502    
Epoch 2/28
200297/200297 [==============================] - 14s - loss: 3.0495    
Epoch 3/28
200297/200297 [==============================] - 13s - loss: 3.0488    
Epoch 4/28
200297/200297 [==============================] - 14s - loss: 3.0480    
Epoch 5/28
200297/200297 [==============================] - 14s - loss: 3.0473    
Epoch 6/28
200297/200297 [==============================] - 13s - loss: 3.0466    
Epoch 7/28
200297/200297 [==============================] - 14s - loss: 3.0458    
Epoch 8/28
200297/200297 [==============================] - 14s - loss: 3.0451    
Epoch 9/28
200297/200297 [==============================] - 13s - loss: 3.0443    
Epoch 10/28
200297/200297 [==============================] - 14s - loss: 3.0436    
Epoch 11/28
200297/200297 [==============================] - 13s - loss: 3.0428    
Epoch 12/28
200297/200297 [==============================] - 14s - loss: 3.0421    
E

In [82]:
model.optimizer.lr=0.0000001

In [83]:
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)

Epoch 1/28
200297/200297 [==============================] - 14s - loss: 3.0290    
Epoch 2/28
200297/200297 [==============================] - 13s - loss: 3.0283    
Epoch 3/28
200297/200297 [==============================] - 13s - loss: 3.0275    
Epoch 4/28
200297/200297 [==============================] - 13s - loss: 3.0267    
Epoch 5/28
200297/200297 [==============================] - 13s - loss: 3.0259    
Epoch 6/28
200297/200297 [==============================] - 13s - loss: 3.0251    
Epoch 7/28
200297/200297 [==============================] - 14s - loss: 3.0243    
Epoch 8/28
200297/200297 [==============================] - 14s - loss: 3.0235    
Epoch 9/28
200297/200297 [==============================] - 14s - loss: 3.0227    
Epoch 10/28
200297/200297 [==============================] - 13s - loss: 3.0219    
Epoch 11/28
200297/200297 [==============================] - 14s - loss: 3.0211    
Epoch 12/28
200297/200297 [==============================] - 14s - loss: 3.0202    
E

In [84]:
model.optimizer.lr=0.01

In [85]:
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)

Epoch 1/28
200297/200297 [==============================] - 14s - loss: 3.0061    
Epoch 2/28
200297/200297 [==============================] - 14s - loss: 3.0053    
Epoch 3/28
200297/200297 [==============================] - 14s - loss: 3.0044    
Epoch 4/28
200297/200297 [==============================] - 13s - loss: 3.0036    
Epoch 5/28
200297/200297 [==============================] - 14s - loss: 3.0027    
Epoch 6/28
200297/200297 [==============================] - 13s - loss: 3.0019    
Epoch 7/28
200297/200297 [==============================] - 14s - loss: 3.0010    
Epoch 8/28
200297/200297 [==============================] - 14s - loss: 3.0002    
Epoch 9/28
200297/200297 [==============================] - 13s - loss: 2.9993    
Epoch 10/28
200297/200297 [==============================] - 14s - loss: 2.9985    
Epoch 11/28
200297/200297 [==============================] - 14s - loss: 2.9976    
Epoch 12/28
200297/200297 [==============================] - 14s - loss: 2.9967    
E

In [90]:
model.optimizer.lr=0.01
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)


Epoch 1/28
200297/200297 [==============================] - 14s - loss: 2.9820    
Epoch 2/28
200297/200297 [==============================] - 13s - loss: 2.9811    
Epoch 3/28
200297/200297 [==============================] - 14s - loss: 2.9802    
Epoch 4/28
200297/200297 [==============================] - 14s - loss: 2.9793    
Epoch 5/28
200297/200297 [==============================] - 13s - loss: 2.9785    
Epoch 6/28
200297/200297 [==============================] - 14s - loss: 2.9776    
Epoch 7/28
200297/200297 [==============================] - 13s - loss: 2.9767    
Epoch 8/28
200297/200297 [==============================] - 14s - loss: 2.9758    
Epoch 9/28
200297/200297 [==============================] - 13s - loss: 2.9749    
Epoch 10/28
200297/200297 [==============================] - 13s - loss: 2.9741    
Epoch 11/28
200297/200297 [==============================] - 14s - loss: 2.9732    
Epoch 12/28
200297/200297 [==============================] - 13s - loss: 2.9723    
E

In [91]:
model.optimizer.lr=0.001
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)

Epoch 1/28
200297/200297 [==============================] - 13s - loss: 2.9574    
Epoch 2/28
200297/200297 [==============================] - 13s - loss: 2.9565    
Epoch 3/28
200297/200297 [==============================] - 13s - loss: 2.9556    
Epoch 4/28
200297/200297 [==============================] - 13s - loss: 2.9548    
Epoch 5/28
200297/200297 [==============================] - 14s - loss: 2.9539    
Epoch 6/28
200297/200297 [==============================] - 14s - loss: 2.9530    
Epoch 7/28
200297/200297 [==============================] - 13s - loss: 2.9522    
Epoch 8/28
200297/200297 [==============================] - 14s - loss: 2.9513    
Epoch 9/28
200297/200297 [==============================] - 13s - loss: 2.9504    
Epoch 10/28
200297/200297 [==============================] - 14s - loss: 2.9496    
Epoch 11/28
200297/200297 [==============================] - 14s - loss: 2.9487    
Epoch 12/28
200297/200297 [==============================] - 14s - loss: 2.9478    
E

In [92]:
model.optimizer.lr=0.01
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)

Epoch 1/28
200297/200297 [==============================] - 13s - loss: 2.9332    
Epoch 2/28
200297/200297 [==============================] - 14s - loss: 2.9324    
Epoch 3/28
200297/200297 [==============================] - 14s - loss: 2.9315    
Epoch 4/28
200297/200297 [==============================] - 13s - loss: 2.9307    
Epoch 5/28
200297/200297 [==============================] - 13s - loss: 2.9298    
Epoch 6/28
200297/200297 [==============================] - 14s - loss: 2.9290    
Epoch 7/28
200297/200297 [==============================] - 14s - loss: 2.9281    
Epoch 8/28
200297/200297 [==============================] - 14s - loss: 2.9273    
Epoch 9/28
200297/200297 [==============================] - 14s - loss: 2.9264    
Epoch 10/28
200297/200297 [==============================] - 14s - loss: 2.9256    
Epoch 11/28
200297/200297 [==============================] - 14s - loss: 2.9247    
Epoch 12/28
200297/200297 [==============================] - 13s - loss: 2.9239    
E

In [93]:
model.optimizer.lr=0.01
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)

Epoch 1/28
200297/200297 [==============================] - 13s - loss: 2.9096    
Epoch 2/28
200297/200297 [==============================] - 14s - loss: 2.9088    
Epoch 3/28
200297/200297 [==============================] - 13s - loss: 2.9080    
Epoch 4/28
200297/200297 [==============================] - 14s - loss: 2.9071    
Epoch 5/28
200297/200297 [==============================] - 13s - loss: 2.9063    
Epoch 6/28
200297/200297 [==============================] - 14s - loss: 2.9055    
Epoch 7/28
200297/200297 [==============================] - 14s - loss: 2.9046    
Epoch 8/28
200297/200297 [==============================] - 14s - loss: 2.9038    
Epoch 9/28
200297/200297 [==============================] - 14s - loss: 2.9030    
Epoch 10/28
200297/200297 [==============================] - 14s - loss: 2.9022    
Epoch 11/28
200297/200297 [==============================] - 14s - loss: 2.9013    
Epoch 12/28
200297/200297 [==============================] - 13s - loss: 2.9005    
E

In [94]:
model.optimizer.lr=0.1
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)

Epoch 1/28
200297/200297 [==============================] - 13s - loss: 2.8866    
Epoch 2/28
200297/200297 [==============================] - 13s - loss: 2.8858    
Epoch 3/28
200297/200297 [==============================] - 14s - loss: 2.8849    
Epoch 4/28
200297/200297 [==============================] - 13s - loss: 2.8841    
Epoch 5/28
200297/200297 [==============================] - 14s - loss: 2.8833    
Epoch 6/28
200297/200297 [==============================] - 13s - loss: 2.8825    
Epoch 7/28
200297/200297 [==============================] - 14s - loss: 2.8817    
Epoch 8/28
200297/200297 [==============================] - 13s - loss: 2.8809    
Epoch 9/28
200297/200297 [==============================] - 13s - loss: 2.8801    
Epoch 10/28
200297/200297 [==============================] - 14s - loss: 2.8793    
Epoch 11/28
200297/200297 [==============================] - 14s - loss: 2.8785    
Epoch 12/28
200297/200297 [==============================] - 14s - loss: 2.8776    
E

In [95]:
model.optimizer.lr =0.1
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)

Epoch 1/28
200297/200297 [==============================] - 14s - loss: 2.8639    
Epoch 2/28
200297/200297 [==============================] - 14s - loss: 2.8631    
Epoch 3/28
200297/200297 [==============================] - 14s - loss: 2.8623    
Epoch 4/28
200297/200297 [==============================] - 14s - loss: 2.8615    
Epoch 5/28
200297/200297 [==============================] - 14s - loss: 2.8607    
Epoch 6/28
200297/200297 [==============================] - 14s - loss: 2.8599    
Epoch 7/28
200297/200297 [==============================] - 14s - loss: 2.8591    
Epoch 8/28
200297/200297 [==============================] - 14s - loss: 2.8583    
Epoch 9/28
200297/200297 [==============================] - 14s - loss: 2.8574    
Epoch 10/28
200297/200297 [==============================] - 14s - loss: 2.8566    
Epoch 11/28
200297/200297 [==============================] - 14s - loss: 2.8558    
Epoch 12/28
200297/200297 [==============================] - 14s - loss: 2.8550    
E

In [96]:
model.optimizer.lr=0.1
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)

Epoch 1/28
200297/200297 [==============================] - 15s - loss: 2.8414    
Epoch 2/28
200297/200297 [==============================] - 13s - loss: 2.8406    
Epoch 3/28
200297/200297 [==============================] - 14s - loss: 2.8398    
Epoch 4/28
200297/200297 [==============================] - 14s - loss: 2.8390    
Epoch 5/28
200297/200297 [==============================] - 14s - loss: 2.8382    
Epoch 6/28
200297/200297 [==============================] - 14s - loss: 2.8374    
Epoch 7/28
200297/200297 [==============================] - 13s - loss: 2.8366    
Epoch 8/28
200297/200297 [==============================] - 14s - loss: 2.8358    
Epoch 9/28
200297/200297 [==============================] - 14s - loss: 2.8350    
Epoch 10/28
200297/200297 [==============================] - 14s - loss: 2.8342    
Epoch 11/28
200297/200297 [==============================] - 13s - loss: 2.8334    
Epoch 12/28
200297/200297 [==============================] - 14s - loss: 2.8326    
E

In [97]:
model.optimizer.lr=0.01
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)

Epoch 1/28
200297/200297 [==============================] - 14s - loss: 2.8190    
Epoch 2/28
200297/200297 [==============================] - 14s - loss: 2.8182    
Epoch 3/28
200297/200297 [==============================] - 14s - loss: 2.8174    
Epoch 4/28
200297/200297 [==============================] - 14s - loss: 2.8166    
Epoch 5/28
200297/200297 [==============================] - 14s - loss: 2.8158    
Epoch 6/28
200297/200297 [==============================] - 14s - loss: 2.8150    
Epoch 7/28
200297/200297 [==============================] - 14s - loss: 2.8142    
Epoch 8/28
200297/200297 [==============================] - 14s - loss: 2.8134    
Epoch 9/28
200297/200297 [==============================] - 14s - loss: 2.8126    
Epoch 10/28
200297/200297 [==============================] - 14s - loss: 2.8118    
Epoch 11/28
200297/200297 [==============================] - 14s - loss: 2.8110    
Epoch 12/28
200297/200297 [==============================] - 14s - loss: 2.8102    
E

In [98]:
model.optimizer.lr=0.0001
model.fit([x1,x2,x3], y, batch_size=64, epochs=28)

Epoch 1/28
200297/200297 [==============================] - 13s - loss: 2.7969    
Epoch 2/28
200297/200297 [==============================] - 13s - loss: 2.7961    
Epoch 3/28
200297/200297 [==============================] - 13s - loss: 2.7953    
Epoch 4/28
200297/200297 [==============================] - 14s - loss: 2.7945    
Epoch 5/28
200297/200297 [==============================] - 14s - loss: 2.7937    
Epoch 6/28
200297/200297 [==============================] - 13s - loss: 2.7930    
Epoch 7/28
200297/200297 [==============================] - 13s - loss: 2.7922    
Epoch 8/28
200297/200297 [==============================] - 14s - loss: 2.7914    
Epoch 9/28
200297/200297 [==============================] - 14s - loss: 2.7906    
Epoch 10/28
200297/200297 [==============================] - 14s - loss: 2.7898    
Epoch 11/28
200297/200297 [==============================] - 13s - loss: 2.7891    
Epoch 12/28
200297/200297 [==============================] - 14s - loss: 2.7883    
E

## Test Model

In [99]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [105]:
get_next('tr ')

't'

In [108]:
get_next('th ')

't'

In [109]:
get_next('an ')

't'